## Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN)

This can be challenging and oftern requires experimentation. However, there are some guidelines and methods that can help you in maiing an informed decision:

- **Start Simple:** Begin with a simple architecture and gradually increase complexity if needed.
- **Grid Search/ Random Search:** Use grid search or random search to try different architectures.
- **Cross Validation:** Use cross-validation to evaluate the performance of different architectures.
- **Heuristics and Rules of Thumnb:** SOme heuristics and empirical rules can provide the starting points, sucah as:

1. The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
2. A common practice is to start with 1-2 hidden layers.

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import pickle

In [30]:
data = pd.read_csv("Churn_Modelling.csv")
data = data.drop(columns=["RowNumber", "CustomerId", "Surname"], axis=1)

label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

onehot_encoder_geography = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geography.fit_transform(data[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geography.get_feature_names_out(["Geography"]))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop("Exited", axis=1)
y = data["Exited"]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('onehot_encoder_geography.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geography, f)    
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [33]:
## Defien a function to create a model adn try different parameters (kerasClassifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [34]:
## Create a KerasClassifier

model = KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50, batch_size=10, verbose=0)

In [35]:
param_grid = {
    'neurons': [16,32,34,128],
    'layers': [1,2],
    'epochs': [50,100]
}

In [36]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

print(f"Best: %f using %s" %(grid_result.best_score_, grid_result.best_params_))

c:\Users\Shardul\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Shardul\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Shardul\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Shardul\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential m

Best: 0.857000 using {'epochs': 100, 'layers': 1, 'neurons': 32}
